### Generating an Interactive Graph Directly from Script

Proof of Concept: CEPR Prices Byte chart generated by using the BLS API to collect data on prices and plotting the data using plotly.

*Brian Dew and Kevin Cashman*

June 14, 2017

In [2]:
# Import libraries 
import requests
import json
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.grid_objs import Column, Grid


import config   ## File with API keys (just to prevent my API keys from being public)
plotly.tools.set_credentials_file(username='bdew', api_key=config.plotly_key)
api_key = config.bls_key

##### Retrieve data from BLS

Request json data from the BLS API for whichever series are of interest.

In [3]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(api_key)
headers = {'Content-type': 'application/json'}
series_dict = {'CUSR0000SA0L1E': 'Core CPI',
               'CUSR0000SA0L12E': 'Without Shelter'}
series_list = series_dict.keys()  # Take just the series names as a list

# Parameters for the API request
data = json.dumps({"seriesid": series_list, "startyear":"2010", "endyear":"2017"})

In [4]:
# Request json data
json_data = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()

In [5]:
# Store the relevant information in a dictionay and combine into a pandas dataframe
d = {}
for series in json_data['Results']['series']:
    seriesId = series['seriesID']
    d[seriesId] = pd.DataFrame(series['data'])
    # Convert BLS API dates to readable format (YYYY-MM-DD)
    d[seriesId]['x'] = pd.to_datetime(d[seriesId]['period'] + ' ' + d[seriesId]['year'])
    # Keep only date and series values
    d[seriesId] = d[seriesId].sort_values(by='x')[['x', 'value']].set_index('x')
    # Take the annual percent change of the index and convert to percent
    d[seriesId] = d[seriesId].astype(float).pct_change(12).multiply(100).dropna()
    # Rename and round
    d[seriesId] = d[seriesId]['value'].rename(series_dict[seriesId]).round(1)
df = pd.concat([d[series_list[0]], d[series_list[1]]], axis=1)

##### View last 5 rows of grid

In [8]:
df.tail()

,Without Shelter,Core CPI
x,,
2017-01-01,1.3,2.3
2017-02-01,1.3,2.2
2017-03-01,1.0,2.0
2017-04-01,0.8,1.9
2017-05-01,0.6,1.7


##### Plotly plot details

This section contains the plotly details which generate the specific plot of interest. One option for getting this information is to use to plotly API to obtain the details of an existing plot in the language of preference. For example, the python code below can be generated through the following link: https://plot.ly/~cashmank/395.py

R code can be generated by changing the file extension to 'r': https://plot.ly/~cashmank/395.r

The uid, xscr and yscr dictionary items contained in each trace are not needed.

Importantly, at this stage we can convert hard coded data values into references to the pandas dataframe generated above. This is what connects the API data with the plotly format.

In [9]:
# Each trace becomes a line.
trace1 = {
  "x": df.index,   # df.index contains the dates
  "y": df[series_dict[series_list[0]]].values,  # without shelter inflation values 
  "line": {"color": "rgb(140, 286, 75)"}, # Adjust R, G, B values for line color here
  "name": series_dict[series_list[0]], # The series name
  "type": "scatter"
}
trace2 = {
  "x": df.index, 
  "y": df[series_dict[series_list[1]]].values, 
  "line": {"color": "rgb(227, 119, 194)"},  # Adjust R, G, B values for line color here
  "name": series_dict[series_list[1]], 
  "type": "scatter"
}
data = Data([trace1, trace2])

# The layout section can also be generated by the script file. However, in this example, it is
#   hard-coded below.
layout = {
  "annotations": [
    {
      "x": -0.0291, 
      "y": 1.1432, 
      "align": "left", 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "Year-over-Year Growth in Core CPI and Core CPI Minus Shelter<br><i>2011 to 2017, Seasonally Adjusted</i>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": -0.0365, 
      "y": -0.1554, 
      "align": "left", 
      "font": {"size": 9}, 
      "showarrow": False, 
      "text": "<a href=\"http://cepr.net>http://cepr.net</a><br>Source: Bureau of Labor Statistics, series <a href=\"https://data.bls.gov/timeseries/CUSR0000SA0L1E\">CUSR0000SA0L1E</a> and <a href=\"https://data.bls.gov/timeseries/CUSR0000SA0L12E\">CUSR0000SA0L12E</a>. Core CPI is all items less food and energy.", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": 0.9703, 
      "y": 0.9432, 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "<b>Core CPI</b>", 
      "xref": "paper", 
      "yref": "paper"
    }, 
    {
      "x": 1.0234, 
      "y": 0.4649, 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "<b>Core CPI without<br>Shelter</b>", 
      "xref": "paper", 
      "yref": "paper"
    }
  ], 
  "autosize": False, 
  "height": 500, 
  "legend": {
    "x": 0.682337662338, 
    "y": 1
  }, 
  "margin": {
    "r": 20, 
    "t": 70, 
    "b": 60, 
    "l": 40
  }, 
  "showlegend": False, 
  "title": "", 
  "width": 700, 
  "xaxis": {
    "autorange": False, 
    "nticks": 8, 
    "range": ["2010-10-01", "2017-08-01"], 
    "showgrid": False, 
    "ticks": "", 
    "title": "", 
    "type": "date", 
    "zeroline": False
  }, 
  "yaxis": {
    "autorange": False, 
    "dtick": 0.5, 
    "exponentformat": "none", 
    "range": [0.6, 2.6], 
    "showgrid": False, 
    "tickmode": "linear", 
    "tickprefix": "", 
    "ticksuffix": "%", 
    "title": "", 
    "type": "linear"
  }
}
fig = Figure(data=data, layout=layout)
# Lastly, to overwrite an existing file (or name a new one) include the filename below
plot_url = py.plot(fig, filename ="prices_byte_example")